<a href="https://colab.research.google.com/github/hyunbini/Project/blob/main/IDSL_LAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library

In [246]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Function used in code

In [247]:
def check_nan(file):
  result = file.isnull().sum()
  return result

In [248]:
#Function that extracts user_id who wrote one or more reviews
#input : series(pandas dataframe)
#output : result(list)
def make_list(series1):
  result = []
  for i in range(series1.size):
    #Add user_id who have written reviews more than once to the new list
    if series1[i]>1:
      result.append(series1.index[i])
  return result

In [249]:
#Function that erases unnecessary characters and makes a list of only the necessary data
#input : list1(list)
#output : result_listset(list)
def remove_special(list1):
  #Change list to string
  for i in range(len(list1)):
    list1[i] = str(list1[i])
  listset_str = ' '.join(list1)
  #Remove Special characters to space
  listset_str = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', "",listset_str)
  #Split the data by space
  result_listset = listset_str.split(' ')
  return result_listset

Import Data and Remove NaN Value and Unuseful Columns



In [250]:
meta_json = pd.read_json('/content/drive/MyDrive/IDSL Recruitment/[Data 1] Eng - Review Analysis/metadata.json')
review_json = pd.read_json('/content/drive/MyDrive/IDSL Recruitment/[Data 1] Eng - Review Analysis/reviews.json')
print(check_nan(meta_json))
print(check_nan(review_json))

main_category          0
title                  0
average_rating         0
rating_number          0
features               0
description            0
price                  0
images                 0
videos                 0
store               1105
categories             0
details                0
parent_asin            0
bought_together    11257
subtitle           11257
author             11257
dtype: int64
rating               0
title                0
text                 0
images               0
asin                 0
parent_asin          0
user_id              0
timestamp            0
helpful_vote         0
verified_purchase    0
dtype: int64


In [251]:
metadata = meta_json.drop(['images', 'videos', 'categories', 'details', 'bought_together', 'subtitle', 'author'],axis = 1)
review = review_json.drop(['title', 'text','images','timestamp','user_id'],axis=1)

Extracting duplicate data and labeling of the user_id value to be used for prediction

In [252]:
id = review_json['user_id']
print(id.size)
id_count = (id.value_counts())
print(id_count.size)
dup_ind = make_list(id_count)

70883
68934


<ipython-input-248-17c3463facae>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if series1[i]>1:


In [253]:
#Function that labels data composed of complex characters numerically based on ascending order\
#input : list1(list to be replaced), list2(reference list), dic(dictionary to store the values of reference list and its unique labeling numbers)
#output : list_df(Dataframe made from a labeled list)
def labeling(list1,list2,dic):
  i = 0
  for data in list1:
    if data in list2:
      if data in dic.keys():
        list1[i] = dic[data]
      else:
        dic[data] = i
        list1[i] = i
      i = i+1
    else:
      list1[i] = i
      i = i+1
  for y in range(len(list1)):
    list1[y] = list1[y]+1
  list_df = pd.DataFrame(data=list1, columns= ['label_id'])
  return list_df

In [254]:
new_user_id = id.to_list()
result_dup_id = remove_special(dup_ind)
result_user_id = remove_special(new_user_id)
label_dic = {}
label_df = labeling(result_user_id,result_dup_id,label_dic)

Modeling

In [255]:
def recommendation_model():

  top3_result = {}
  for parent_asin, rating in result() :


SyntaxError: incomplete input (<ipython-input-255-89ce78fab6f1>, line 4)